In [ ]:
#상황을 입력받고 옷 추천 & 생년월일 입력받고 행운의 색 & 상황과 행운의 색을 반영하여 옷 추천

import ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

api_key = "YOUR_OLLAMA_API_KEY"
chat_model = ChatOllama(api_key=api_key, model='llama3.1')

outfit_prompt_template = ChatPromptTemplate.from_template(
    template="당신은 패션 전문가입니다. {occasion}에 어울리는 옷을 간단하게 추천해 주세요. (예: 정장, 구두)"
)

color_prompt_template = ChatPromptTemplate.from_template(
    template="{birthdate}에 태어난 사람에게 오늘의 행운의 색을 간단하게 알려 주세요. (예: 빨간색, 파란색)"
)

lucky_color_outfit_prompt_template = ChatPromptTemplate.from_template(
    template=(
        "당신은 패션 전문가입니다. {occasion}에 어울리는 옷을 간단하게 추천해 주세요. "
        "특히 오늘의 행운의 색인 {lucky_color}를 반영해 주세요. (예: 노란색 넥타이, 파란색 셔츠)"
    )
)

output_parser = StrOutputParser()

def recommend_outfit(occasion):
    prompt = outfit_prompt_template.format_prompt(occasion=occasion)
    response = chat_model(prompt.to_messages())
    outfit_recommendation = output_parser.parse(response)
    return outfit_recommendation.content

def recommend_lucky_color(birthdate):
    prompt = color_prompt_template.format_prompt(birthdate=birthdate)
    response = chat_model(prompt.to_messages())
    color_recommendation = output_parser.parse(response)
    return color_recommendation.content

def recommend_outfit_with_lucky_color(occasion, lucky_color):
    prompt = lucky_color_outfit_prompt_template.format_prompt(occasion=occasion, lucky_color=lucky_color)
    response = chat_model(prompt.to_messages())
    outfit_recommendation = output_parser.parse(response)
    return outfit_recommendation.content 

occasion = input("어떤 상황에 맞는 옷을 추천받고 싶으세요? (예: 데이트, 면접 등): ")
birthdate = input("생년월일을 입력하세요 (YYYY-MM-DD): ")

outfit = recommend_outfit(occasion)
print(f"추천하는 옷: {outfit}")

lucky_color = recommend_lucky_color(birthdate)
print(f"오늘의 행운의 색: {lucky_color}")

use_lucky_color = input("행운의 색을 반영한 옷 추천을 받고 싶으세요? (예/아니요): ").strip().lower()
if use_lucky_color == "예":
    outfit_with_lucky_color = recommend_outfit_with_lucky_color(occasion, lucky_color)
    print(f"행운의 색을 반영한 추천 옷: {outfit_with_lucky_color}")
else:
    print("기본 추천 옷을 사용합니다.")

C:\Users\miso3\AppData\Local\Temp\ipykernel_3632\1709445667.py:37: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat_model(prompt.to_messages())


추천하는 옷: 운동에는 다양한 종류가 있지만, 일반적으로는 활동성이 높고 편안한 옷이 좋습니다.

1.  **자켓과 팬츠**: 활동적인 날씨에 적합하며, 운동 시도 또는 레저 활동에 용이합니다.
2.  **스포츠 바지**: 트레이닝을 위한 것이나, 산책이나 스포츠를 좋아하는 사람들에게 적합합니다.
3.  **데님팬츠와 스니커즈**: 일반적으로 주말 나누어지는 시간에 더 많이 착용하는 옷입니다.
오늘의 행운의 색: 당신의 행운의 색은 녹색입니다.
행운의 색을 반영한 추천 옷: 운동에 적합한 녹색 옷 추천입니다.

1.  **녹색 운동복**: 가장 간단한 방법입니다. 녹색의 유니폼과 레그린저를 장착하면 운동하는 동안도 멋스럽습니다.
2.  **녹색 반바지**: 반바지로 운동을 하려는 경우, 녹색 반바지를 추천합니다. 
3.  **녹색 티셔츠**: 티셔츠로 운동을 하려고 한다면, 녹색 티셔츠를 선택하면 됩니다.


In [ ]:
#상황을 입력받고 옷 추천 & 생년월일 입력받고 행운의 색 -> 이미지 파일 저장(실패)

import os
from langchain_community.chat_models import ChatOllama
from PIL import Image, ImageDraw, ImageFont
from langchain_core.prompts import ChatPromptTemplate

api_key = "YOUR_OLLAMA_API_KEY"
chat_model = ChatOllama(api_key=api_key, model='llama3.1')

def recommend_outfit(occasion):
    prompt = ChatPromptTemplate.from_template(
        template="당신은 패션 전문가입니다. {occasion}에 어울리는 옷을 간단하게 추천해 주세요. (예: 정장, 구두)"
    ).format_prompt(occasion=occasion)
    
    response = chat_model.invoke(prompt.to_messages())
    outfit_recommendation = response.content 
    return outfit_recommendation

def recommend_lucky_color(birthdate):
    prompt = ChatPromptTemplate.from_template(
        template="{birthdate}에 태어난 사람에게 오늘의 행운의 색을 간단하게 알려 주세요. (예: 빨간색, 파란색)"
    ).format_prompt(birthdate=birthdate)
    response = chat_model.invoke(prompt.to_messages())
    color_recommendation = response.content.strip()
    
    if "죄송하지만" in color_recommendation:
        color_recommendation = "알 수 없음" 
    
    return color_recommendation

def generate_outfit_image(occasion, lucky_color):
    prompt = f"당신은 패션 전문가입니다. {occasion}에 어울리는 옷을 간단하게 묘사해 주세요. 특히 오늘의 행운의 색인 {lucky_color}를 반영해 주세요."
    response = chat_model.invoke(prompt)
    outfit_description = response.content.strip()

    image = Image.new('RGB', (600, 400), (255, 255, 255))
    draw = ImageDraw.Draw(image)
    
    try:
        font = ImageFont.truetype('arial.ttf', size=24)
    except IOError:
        font = ImageFont.load_default()

    x, y = 50, 50
    draw.text((x, y), outfit_description, font=font, fill=(0, 0, 0))
    
    safe_color = lucky_color.replace(" ", "_").replace("/", "_")
    safe_occasion = occasion.replace(" ", "_").replace("/", "_")
    image_filename = f"{safe_occasion}_{safe_color}_outfit.png"

    save_path = os.path.join(os.getcwd(), image_filename)
    
    image.save(save_path)
    print(f"이미지가 '{image_filename}'에 저장되었습니다.")
    
    return save_path

occasion = input("어떤 상황에 맞는 옷을 추천받고 싶으세요? (예: 데이트, 면접 등): ")
birthdate = input("생년월일을 입력하세요 (YYYY-MM-DD): ")

outfit = recommend_outfit(occasion)
print(f"추천하는 옷: {outfit}")

lucky_color = recommend_lucky_color(birthdate)
print(f"오늘의 행운의 색: {lucky_color}")

save_image = input("추천 옷 이미지를 저장하시겠습니까? (예/아니요): ").strip().lower()
if save_image == "예":
    image_filename = generate_outfit_image(occasion, lucky_color)
    print(f"이미지가 '{image_filename}'에 저장되었습니다.")
else:
    print("이미지는 저장하지 않습니다.")


추천하는 옷: 운동에 어울리는 옷으로는 신축성 좋은 니트 원피스와 긴팔 셔츠, 스타킹과 슬랙스 등이 있습니다.
오늘의 행운의 색: 파란색입니다.
이미지가 '운동_파란색입니다._outfit.png'에 저장되었습니다.
이미지가 'c:\Users\miso3\OneDrive\바탕 화면\gdg\ai과제2\운동_파란색입니다._outfit.png'에 저장되었습니다.
